# ISAM

`gtsam.ISAM` (Incremental Smoothing and Mapping) is a class that inherits from `BayesTree` and adds an `update` method. This method allows for efficient incremental updates to the solution when new factors (e.g., new measurements) are added to the problem.

Instead of re-eliminating the entire factor graph from scratch, iSAM identifies the part of the Bayes tree affected by the new factors, removes that part, and re-eliminates only the necessary variables, merging the results back into the existing tree.

Like `BayesTree`, it's templated (e.g., `GaussianISAM` which inherits from `GaussianBayesTree`). For practical applications requiring incremental updates, `ISAM2` is often preferred due to further optimizations like fluid relinearization and support for variable removal, but `ISAM` demonstrates the core incremental update concept based on the Bayes tree.

<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/gtsam/inference/doc/ISAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install gtsam

In [ ]:
import gtsam
import numpy as np

# Use Gaussian variants for demonstration
from gtsam import GaussianFactorGraph, Ordering, GaussianISAM, GaussianBayesTree
from gtsam import symbol_shorthand

X = symbol_shorthand.X
L = symbol_shorthand.L

## Initialization

An `ISAM` object can be created empty or initialized from an existing `BayesTree`.

In [ ]:
# Create an empty ISAM object
isam1 = GaussianISAM()
print("Initial ISAM (empty):")
isam1.print()

# Create from an existing Bayes Tree (e.g., from an initial batch solve)
initial_graph = GaussianFactorGraph()
model = gtsam.noiseModel.Isotropic.Sigma(1, 1.0)
initial_graph.add(X(0), -np.eye(1), np.zeros(1), model) # Prior on x0

initial_bayes_tree = initial_graph.eliminateMultifrontal(Ordering([X(0)]))
print("Initial BayesTree:")
initial_bayes_tree.print()

isam2 = GaussianISAM(initial_bayes_tree)
print("ISAM from BayesTree:")
isam2.print()

Initial ISAM (empty):
: cliques: 0, variables: 0

Initial BayesTree:
: cliques: 1, variables: 1
Root(s):
Conditional density P(x0) = P(x0) 
  size: 1
  Conditional P(x0): GaussianConditional( P(x0) = dx0 - d), d = [ 0 ], sigmas = [ 1 ])


ISAM from BayesTree:
: cliques: 1, variables: 1
Root(s):
Conditional density P(x0) = P(x0) 
  size: 1
  Conditional P(x0): GaussianConditional( P(x0) = dx0 - d), d = [ 0 ], sigmas = [ 1 ])




## Incremental Update

The core functionality is the `update(newFactors)` method.

In [ ]:
# Start with the ISAM object containing the prior on x0
isam = GaussianISAM(initial_bayes_tree)
model = gtsam.noiseModel.Isotropic.Sigma(1, 1.0)

# --- First Update ---
new_factors1 = GaussianFactorGraph()
new_factors1.add(X(0), -np.eye(1), X(1), np.eye(1), np.zeros(1), model) # x0 -> x1
isam.update(new_factors1)

print("ISAM after first update (x0, x1):")
isam.print()

# --- Second Update ---
new_factors2 = GaussianFactorGraph()
new_factors2.add(X(1), -np.eye(1), X(2), np.eye(1), np.zeros(1), model) # x1 -> x2
isam.update(new_factors2)

print("\nISAM after second update (x0, x1, x2):")
isam.print()

ISAM after first update (x0, x1):
: cliques: 2, variables: 2
Root(s):
Conditional density P(x0 | x1) = P(x0 | x1) 
  size: 1
  Conditional P(x0 | x1): GaussianConditional( P(x0 | x1) = dx0 - R*dx1 - d), R = [ 0.5 ], d = [ 0 ], sigmas = [ 0.707107 ])


  Children:
  Conditional density P(x1) = P(x1) 
    size: 1
    Conditional P(x1): GaussianConditional( P(x1) = dx1 - d), d = [ 0 ], sigmas = [ 0.707107 ])



ISAM after second update (x0, x1, x2):
: cliques: 3, variables: 3
Root(s):
Conditional density P(x0 | x1) = P(x0 | x1) 
  size: 1
  Conditional P(x0 | x1): GaussianConditional( P(x0 | x1) = dx0 - R*dx1 - d), R = [ 0.5 ], d = [ 0 ], sigmas = [ 0.707107 ])


  Children:
  Conditional density P(x1 | x2) = P(x1 | x2) 
    size: 1
    Conditional P(x1 | x2): GaussianConditional( P(x1 | x2) = dx1 - R*dx2 - d), R = [ 0.666667 ], d = [ 0 ], sigmas = [ 0.816497 ])


    Children:
    Conditional density P(x2) = P(x2) 
      size: 1
      Conditional P(x2): GaussianConditional( P(x2) = dx2 -

## Solution and Marginals

Since `ISAM` inherits from `BayesTree`, you can use the same methods like `optimize()` and `marginalFactor()` after performing updates.

In [ ]:
# Get the solution from the final ISAM state
solution = isam.optimize()
print("Optimized Solution after updates:")
solution.print()

Optimized Solution after updates:
Values with 3 values:
Value x0: [0.]
Value x1: [0.]
Value x2: [0.]

